In [1]:
import pandas as pd

In [2]:
tmap = pd.read_csv('../data/Tmap_2005.csv')

In [3]:
c1 = tmap['지역(시도)'] == '서울'
select_list = ['공공기관', '의료시설', '연구기관', '업종별기업', '행정기관', '종교', '가정의례']
# c2 = tmap['지역(시군구)'] == '강남구'
# c2 = tmap['일자']==20200510
seoul = tmap.loc[c1,:]
seoul.shape

(23250, 8)

In [4]:
l = []
for i in select_list:
    select_condition2 = tmap['검색지유형2']==i
    seoul_index = seoul.loc[select_condition2,:].index
    # l.append(seoul_index)
    l.extend(seoul_index)
new_seoul = seoul.drop(l)
new_seoul['검색지유형3'].unique()
# seoul['검색지유형2'].unique()
# seoul['검색지유형3'].unique()

array(['아파트브랜드', '지하철역', '아파트단지', '백화점', '아파트건설사', '공원', '할인점', '주상복합',
       '농수축산물시장', '대학교', '쇼핑센터', '테마파크', '주요빌딩', '일반상가', '산업시설기타',
       '고속도로 IC', '오피스텔', '스케이트장', '기차역', '전망대', '주차장', '박물관/기념관', '호텔',
       '전자제품양판점', '도서관', '극장', '커피전문점', '고등학교', '종합시장', '서적브랜드', '전문상가',
       '한식', '미술관', '드라이브코스', '문화유적지', '고궁', '버스터미널', '초등학교', '전시장',
       '수족관', '부페', '공항', '농수축산물점', '교차로', '학교내시설물', '유원지', '재래시장', '방송사',
       '운동장/체육관', '슈퍼마켓', '공연장', '섬', '분식', '기념비', '중학교', '아파트형공장',
       '종합스포츠센타', '골프장', '대한상공회의소', '은행', '주요건물기타', '주유소', '자동차영업소',
       '중고차매매', '대교/철교', '사회복지시설', '화랑/전시관', '삼성', '센터', '능묘', '회관',
       '도시고속화도로IC', '축제', '운전면허시험장', 'LG유플러스', '호수', '한샘가구', '관광안내소',
       '장난감', '동식물원', '패밀리레스토랑', '놀이시설'], dtype=object)

In [11]:
# 서울시 구
gu_list = new_seoul['지역(시군구)'].unique()
# 점수주기
new_seoul['point'] = 31-new_seoul['검색지랭킹(시군구내)']
# 지역별 점수 합산
result = pd.DataFrame()
for i in gu_list:
    con_gu = new_seoul['지역(시군구)'] == i
    # print(i)
    gu_seoul = new_seoul.loc[con_gu, :]
    s = gu_seoul.groupby(by=['검색지명']).sum()
    s = s.loc[s['point']>20,:]
    ss = s.sort_values(by='point', ascending=False).reset_index()
    ss['gu'] = i
    result=result.append(ss[['검색지명', 'point', 'gu']])
result.head(40)


,검색지명,point,gu
0,고덕래미안힐스테이트아파트,930,강동구
1,천호역,904,강동구
2,고덕그라시움아파트,868,강동구
3,강동롯데캐슬퍼스트아파트,795,강동구
4,현대백화점천호점,770,강동구
5,고덕아르테온아파트,727,강동구
6,고덕롯데캐슬베네루체아파트,697,강동구
7,암사선사현대아파트,650,강동구
8,삼익그린2차아파트,632,강동구
9,프라이어팰리스아파트,592,강동구


# 카카오 api
- 카카오 api 키워드로 장소검색을 사용하면 각각의 네비게이션 목적지들의 좌표값이 나온다.
- 지하철 역일경우에는 인구 분포가 안나오는데 그외에는 나온다.

In [6]:
name = result.iloc[0,0]

In [12]:
# cnt_df.plot(kind='bar')
import requests
from decouple import config
ka_key = config('KAKAO_RESTAPI')
kakao_url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query='+name
h = {
    'Authorization': 'KakaoAK '+ka_key
}
res = requests.get(kakao_url, headers=h)
location_data = res.json()['documents']
# 구, 동 따로 구별해서 저장해주기
address = location_data[0]['address_name']
place_url = location_data[0]['place_url']
place_x = location_data[0]['x']
place_y = location_data[0]['y']
# print(place_x, place_y, address)
# location_data

In [8]:
add_sp=address.split()
place_dong, place_gu = add_sp[2], add_sp[1]
print(place_gu, place_dong)

강동구 고덕동


In [13]:
kakao_category_url = f'https://dapi.kakao.com/v2/local/search/category.json?category_group_code=FD6&x={place_x}&y={place_y}&radius=300&page=1&sort=distance'
cate_res = requests.get(kakao_category_url, headers=h)
# cate_res.json()

In [10]:
len(cate_res.json()['documents'])

15

# 만들기

In [20]:
# 서울에 있는 내비게이션 목적지
result
gu_list = result['gu'].unique()

array(['강동구', '성동구', '송파구', '양천구', '영등포구', '용산구', '종로구', '중랑구', '강남구',
       '강서구', '관악구', '광진구', '동작구', '마포구', '서초구', '은평구', '강북구', '구로구',
       '노원구', '도봉구', '동대문구', '성북구', '중구', '금천구', '서대문구'], dtype=object)

In [36]:
navi_list = []
for idx, gu_name in enumerate(gu_list):
    if idx: break  ## test용으로
    con = result['gu'] == gu_name
    location = result.loc[con,:]
    for i in location['검색지명']:
        kakao_url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + i
        res = requests.get(kakao_url, headers=h)
        ### 내비게이션 목적지에서 많이 나오는 애들 항목임.
        location_data = res.json()['documents']
        place_x = location_data[0]['x']
        place_y = location_data[0]['y']
        navi_list.append(
            {
                'name':i,
                'x':place_x,
                'y':place_y,
            }
        )

In [37]:
category = ['']

[{'name': '고덕래미안힐스테이트아파트', 'x': '127.149057394089', 'y': '37.5595258727289'},
 {'name': '천호역', 'x': '127.12392845044', 'y': '37.5385112120297'},
 {'name': '고덕그라시움아파트', 'x': '127.16364074756201', 'y': '37.55754058724754'},
 {'name': '강동롯데캐슬퍼스트아파트', 'x': '127.1430397571237', 'y': '37.55662489972904'},
 {'name': '현대백화점천호점', 'x': '127.1244432563588', 'y': '37.538841968226784'},
 {'name': '고덕아르테온아파트', 'x': '127.16845742861217', 'y': '37.55497948814207'},
 {'name': '고덕롯데캐슬베네루체아파트', 'x': '127.1678007705208', 'y': '37.549312250137085'},
 {'name': '암사선사현대아파트', 'x': '127.1266717599396', 'y': '37.55224672809914'},
 {'name': '삼익그린2차아파트', 'x': '127.14854472216015', 'y': '37.55269966641881'},
 {'name': '프라이어팰리스아파트', 'x': '127.13949839212972', 'y': '37.55352610318784'},
 {'name': '고덕리엔파크3단지아파트', 'x': '127.17552532997468', 'y': '37.55268516575942'},
 {'name': '고덕아이파크아파트', 'x': '127.15524028249118', 'y': '37.55893045558401'},
 {'name': '명일역', 'x': '127.14404374474', 'y': '37.5513978958277'},
 {'name': 